<a href="https://colab.research.google.com/github/amhaiskar0921/AmazonProject/blob/main/main_clean.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/drive')

import pyarrow.parquet as pq
import pandas as pd
import numpy as np

# For data visualization
import seaborn as sns

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [10]:
np.random.seed(42)

sample_size = 100000

shopping_data = pq.read_table('/content/drive/MyDrive/Amazon (LA) - Multi-Class Product Classification (Team A)/Datasets/shopping_queries_dataset_examples.parquet')
df = shopping_data.to_pandas().sample(n=sample_size, random_state=42)
shopping_data_p = pq.read_table('/content/drive/MyDrive/Amazon (LA) - Multi-Class Product Classification (Team A)/Datasets/shopping_queries_dataset_products.parquet')
df_p = shopping_data_p.to_pandas().sample(n=sample_size, random_state=42)
df_p = df_p.drop(columns=['product_locale']) #this is a duplicate from shopping_data

# merging
df_merged = pd.merge(df, df_p, on='product_id', how='inner')
print(f"number of products {len(pd.unique(df_merged['product_id']))}")
df_merged_no_null = df_merged.fillna("")
missing_values = df_merged_no_null.isnull().sum()
print(len(df_merged_no_null))

number of products 5319
5548


In [12]:
# Class imbalance exists:
class_counts = df_merged['esci_label'].value_counts()
print(class_counts)

# We will use the Micro averaging F1 Score, because the four classes are unbalanced:
# 65.17% Exacts, 21.91% Substitutes, 2.89% Complements and 10.04% Irrelevants; and this metric
# is robust enough for this situation.

E    3606
S    1226
I     569
C     147
Name: esci_label, dtype: int64


In [4]:
# concat dat
!pip install transformers
import torch
import transformers
from torch.utils.data import DataLoader, Dataset
from transformers import AutoTokenizer, AutoModel
from transformers import BertTokenizer, BertModel, AdamW, get_linear_schedule_with_warmup, AutoModelForSequenceClassification
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
import pandas as pd

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 45.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 29.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 93.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 70.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 32.7 MB/s eta 0:00:00


In [5]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [27]:
import urllib
from urllib.request import urlopen

# We looked at the repo with japanese stopwords, and they gave instructions to download it in the shell
# in JavaScript.
# We looked into how to do the same thing in Python, and came across this stack overflow post: https://stackoverflow.com/questions/72149806/exclude-japanese-stopwords-from-file
# Following the code from there:
def get_japanese_stopwords():
  # print("in function")
  iso_path = "https://raw.githubusercontent.com/stopwords-iso/stopwords-ja/master/stopwords-ja.txt"
  iso_file = urllib.request.urlopen(iso_path)
  # print(iso_file)
  stopwords = [line.decode("utf-8").strip() for line in iso_file]

  stopwords = [ss for ss in stopwords if not ss==u'']
  stopwords = set(stopwords)
  # print(len(stopwords))
  return stopwords


In [29]:
def get_all_stopwords():
  stop_words_english = set(stopwords.words('english'))
  stop_words_spanish = set(stopwords.words('spanish'))
  stop_words_japanese = get_japanese_stopwords()
  return stop_words_english, stop_words_spanish, stop_words_japanese

# Add this code before tokenization
def remove_stopwords(text, stop_words_english, stop_words_spanish, stop_words_japanese):
    words = text.split()
    filtered_words = [word for word in words if word.lower() not in stop_words_english and word.lower()
    not in stop_words_spanish and word.lower() not in stop_words_japanese]
    return ' '.join(filtered_words)

In [31]:
# Initialize an empty list to store the modified input sequences
input_sequences = []
stop_words_english, stop_words_spanish, stop_words_japanese = get_all_stopwords()

# Apply stopword removal to each element in the 'input_sequences' list using a for loop
for text in df_merged_no_null.apply(lambda x: f"[CLS] {x['query']} [SEP] {x['product_title']} [SEP] {x['product_description']} [SEP] {x['product_bullet_point']} [SEP] {x['product_color']} [SEP] {x['product_bullet_point']} [SEP]", axis=1).tolist():
# for text in df_merged_no_null.apply(lambda x: f"[CLS] {x['product_title']} [SEP] {x['product_description']} [SEP] {x['product_bullet_point']} [SEP]", axis=1).tolist():
    cleaned_text = remove_stopwords(text, stop_words_english, stop_words_spanish, stop_words_japanese)
    input_sequences.append(cleaned_text)
print(input_sequences[0], len(input_sequences))

[CLS] yunsey tinte amoniaco [SEP] LIGHT IRRIDIANCE OXIGENADA 20VOL 1000ML L.I, Negro, 1000mL-1L [SEP] [SEP] OXIGENADA 20VOL 1000ML L.I. [SEP] Negro [SEP] OXIGENADA 20VOL 1000ML L.I. [SEP] 5548


In [32]:
# Load the BERT tokenizer
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

# Example text to tokenize
# text = "This is an example sentence."

# Tokenize the text
tokens = tokenizer(input_sequences, padding=True, truncation=True, return_tensors="pt")

# Convert tokenized text to PyTorch tensors
input_ids = tokens['input_ids']
attention_mask = tokens['attention_mask']

In [ ]:
from torch.utils.data import DataLoader, TensorDataset
def get_data_loader(features, max_seq_length, batch_size, shuffle=True):

    all_input_ids = torch.tensor([f.input_ids for f in features], dtype=torch.long)
    all_input_mask = torch.tensor([f.input_mask for f in features], dtype=torch.long)
    all_segment_ids = torch.tensor([f.segment_ids for f in features], dtype=torch.long)
    all_label_ids = torch.tensor([f.label_id for f in features], dtype=torch.long)
    data = TensorDataset(all_input_ids, all_input_mask, all_segment_ids, all_label_ids)

    dataloader = DataLoader(data, shuffle=shuffle, batch_size=batch_size)
    return dataloader
class BertInputItem(object):
  """An item with all the necessary attributes for finetuning BERT."""

  def __init__(self, text, input_ids, input_mask, segment_ids, label_id):
      self.text = text
      self.input_ids = input_ids
      self.input_mask = input_mask
      self.segment_ids = segment_ids
      self.label_id = label_id


In [ ]:
def convert_examples_to_inputs(example_texts, example_labels, label2idx, max_seq_length, tokenizer, verbose=0):
      """Loads a data file into a list of `InputBatch`s."""

      input_items = []
      examples = zip(example_texts, example_labels)
      for (ex_index, (text, label)) in enumerate(examples):

          # Create a list of token ids
          input_ids = tokenizer.encode(f"[CLS] {text} [SEP]")
          if len(input_ids) > max_seq_length:
              input_ids = input_ids[:max_seq_length]

          # All our tokens are in the first input segment (id 0).
          segment_ids = [0] * len(input_ids)

          # The mask has 1 for real tokens and 0 for padding tokens. Only real
          # tokens are attended to.
          input_mask = [1] * len(input_ids)

          # Zero-pad up to the sequence length.
          padding = [0] * (max_seq_length - len(input_ids))
          input_ids += padding
          input_mask += padding
          segment_ids += padding

          assert len(input_ids) == max_seq_length
          assert len(input_mask) == max_seq_length
          assert len(segment_ids) == max_seq_length

          label_id = label2idx[label]

          input_items.append(
              BertInputItem(text=text,
                            input_ids=input_ids,
                            input_mask=input_mask,
                            segment_ids=segment_ids,
                            label_id=label_id))


      return input_items

In [ ]:
rest_texts, test_texts, rest_labels, test_labels = train_test_split(input_sequences, df_merged_no_null['esci_label'].tolist(), test_size=0.1, random_state=1)
train_texts, dev_texts, train_labels, dev_labels = train_test_split(rest_texts, rest_labels, test_size=0.1, random_state=1)

target_names = list(set(df_merged_no_null['esci_label'].tolist()))
label2idx = {label: idx for idx, label in enumerate(target_names)}
print(label2idx)

{'C': 0, 'S': 1, 'E': 2, 'I': 3}


In [ ]:
# inputItem = BertInputItem()
MAX_SEQ_LENGTH = 512
train_features = convert_examples_to_inputs(train_texts, train_labels, label2idx, MAX_SEQ_LENGTH, tokenizer, verbose=0)
dev_features = convert_examples_to_inputs(dev_texts, dev_labels, label2idx, MAX_SEQ_LENGTH, tokenizer)
test_features = convert_examples_to_inputs(test_texts, test_labels, label2idx, MAX_SEQ_LENGTH, tokenizer)

BATCH_SIZE = 16

train_dataloader = get_data_loader(train_features, MAX_SEQ_LENGTH, BATCH_SIZE, shuffle=True)
dev_dataloader = get_data_loader(dev_features, MAX_SEQ_LENGTH, BATCH_SIZE, shuffle=False)
test_dataloader = get_data_loader(test_features, MAX_SEQ_LENGTH, BATCH_SIZE, shuffle=False)

Token indices sequence length is longer than the specified maximum sequence length for this model (808 > 512). Running this sequence through the model will result in indexing errors


In [ ]:
df_merged_no_null.shape

(10220, 14)

In [ ]:
# creating custom model with numerical features
model = AutoModelForSequenceClassification.from_pretrained("bert-base-cased")
additional_features = df_merged_no_null[["large_version", "small_version"]] #should we include this in the model?

#work in progress, need to create base model
import torch.optim as optim
from torch import nn

EPOCHS = 1

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
for epoch in range(EPOCHS):
    model.train()
    for batch in train_dataloader:
        text_inputs, numerical_inputs, labels = batch

        # Clear gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = model(text_inputs, numerical_inputs)

        # Calculate loss
        loss = criterion(outputs, labels)

        # Backpropagation
        loss.backward()
        optimizer.step()
